<a href="https://colab.research.google.com/github/heathjohn62/CS155-Fake-Deep/blob/main/project2/get_UV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Miniproject 2: MovieLens
Authors: Julia Sloan, Ayooluwa Odemuyiwa, Randall Pulido, John Heath

In [1]:
import numpy as np
import matplotlib.pyplot as plt

Fill in these functions to train the SVD

In [2]:
def grad_U(Ui, Yij, Vj, reg, eta):
    """
    Takes as input Ui (the ith row of U), a training point Yij, the column
    vector Vj (jth column of V^T), reg (the regularization parameter lambda),
    and eta (the learning rate).

    Returns the gradient of the regularized loss function with
    respect to Ui multiplied by eta.
    """
    grad = reg * Ui - (Yij - np.dot(Ui.T, Vj)) * Vj
    return eta * grad


def grad_V(Vj, Yij, Ui, reg, eta):
    """
    Takes as input the column vector Vj (jth column of V^T), a training point Yij,
    Ui (the ith row of U), reg (the regularization parameter lambda),
    and eta (the learning rate).

    Returns the gradient of the regularized loss function with
    respect to Vj multiplied by eta.
    """
    grad = reg * Vj - (Yij - np.dot(Ui.T, Vj)) * Ui
    return eta * grad

def get_err(U, V, Y, reg=0.0):
    """
    Takes as input a matrix Y of triples (i, j, Y_ij) where i is the index of a user,
    j is the index of a movie, and Y_ij is user i's rating of movie j and
    user/movie matrices U and V.

    Returns the mean regularized squared-error of predictions made by
    estimating Y_{ij} as the dot product of the ith row of U and the jth column of V^T.
    """
    u_norm = np.linalg.norm(U)
    v_norm = np.linalg.norm(V)
    norms = (reg / 2.0) * ((u_norm ** 2) + (v_norm ** 2))

    error = norms
    for y in Y:
        y_ij = y[2]
        i = y[0] - 1
        j = y[1] - 1

        est = np.dot(U[i], V[j])
        error += .5 * ((y_ij - est) ** 2)

    return error / len(Y)


def train_model(M, N, K, eta, reg, Y, eps=0.0001, max_epochs=300):
    """
    Given a training data matrix Y containing rows (i, j, Y_ij)
    where Y_ij is user i's rating on movie j, learns an
    M x K matrix U and N x K matrix V such that rating Y_ij is approximated
    by (UV^T)_ij.

    Uses a learning rate of <eta> and regularization of <reg>. Stops after
    <max_epochs> epochs, or once the magnitude of the decrease in regularized
    MSE between epochs is smaller than a fraction <eps> of the decrease in
    MSE after the first epoch.

    Returns a tuple (U, V, err) consisting of U, V, and the unregularized MSE
    of the model.
    """
    # Initialize U and V to contain small random numbers between -.5 and .5
    U = np.random.rand(M, K) - 0.5
    V = np.random.rand(N, K) - 0.5

    NUM_EPOCHS = 300
    error0 = get_err(U, V, Y, reg)
    prev_error = error0
    error = prev_error + 5.0    
    
    # Loop over defined number of epochs
    for n in range(NUM_EPOCHS):
        # Loop over all points in Y randomly
        indices = np.random.permutation(len(Y))
        for m in indices:
            i = Y[m][0] - 1
            j = Y[m][1] - 1
            y = Y[m][2]

            U[i] -= grad_U(U[i], y, V[j], reg, eta)
            V[j] -= grad_V(V[j], y, U[i], reg, eta)

        
        error = get_err(U, V, Y, reg)
        if n == 0:
            error1 = error

        if ((np.abs(prev_error - error) / np.abs(error1 - error0)) <= eps):
            return (U, V, prev_error)

        prev_error = error

    return (U, V, prev_error)

Run the cell below to find U and V

In [3]:
# Data columns: User ID, Movie ID, Rating
Y_train = np.loadtxt('https://raw.githubusercontent.com/lakigigar/Caltech-CS155-2021/main/projects/project2/data/train.txt').astype(int)
Y_test = np.loadtxt('https://raw.githubusercontent.com/lakigigar/Caltech-CS155-2021/main/projects/project2/data/test.txt').astype(int)

M = max(max(Y_train[:,0]), max(Y_test[:,0])).astype(int) # users
N = max(max(Y_train[:,1]), max(Y_test[:,1])).astype(int) # movies
print("Factorizing with ", M, " users, ", N, " movies.")

# Use k=20 as specified in project specs
K = 20

reg = 0.0
eta = 0.03 # learning rate
E_in = []
E_out = []

# Use to compute Ein and Eout
U,V, err = train_model(M, N, K, eta, reg, Y_train)
E_in = err
E_out = get_err(U, V, Y_test)

Factorizing with  943  users,  1682  movies.
